# Top статей
## Часть системы, отвечающая за составление топа статей для тех, у кого еще нет истории взаимодействия

## Задача: выделить из истории взаимодействий пользователей те статьи, куда чаще всего кликают пользователи определенного возраста/пола
1) Предлагается следующее разделение
    - '0-17'
    - '18-29'
    - '30-44'
    - '45-59'
    - '60+'
2) После подсчета нужно куда-то сохранить результаты
3) Пересчет раз в неделю

In [2]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight

%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'
pd.set_option('display.max_columns', 50)  # Показывать до 50 колонок
pd.set_option('display.max_colwidth', 100)  # Показывать до 50 колонок

In [3]:
data = pd.read_excel("../cuprum_3.xlsx", sheet_name="Лист4") #cuprum/Лист4/Лист2/Лист1
data

,id,action_type,article_id,ehr_id,created_at,medialog_ehr_id,esb_ehr_id,patientnet_ehr_id,пол,возраст,birthday,title,url,views,published_date,formats,tags,rubric_title
0,3744933,CLICKED,61c03691a7568f6cda4a0c95,961420,2023-07-01,961420,175881,ed4cb6d0-fc26-43f7-8322-292adc7e6f09,2,35,1990-05-29,Чем опасна трипофобия,https://cuprum.media/science-answers/trypophobia?from=avaapp,795.0,2021-12-20,longread,0,Ответили по науке
1,3745298,CLICKED,5f22ce85e24e57105a57d2a5,100445,2023-07-01,100445,38203,703c46d2-b15f-4d00-b4da-7698eca126a3,1,37,1987-09-26,Мирамистин и хлоргексидин можно использовать вместо санитайзера?,https://cuprum.media/proverka-sluha/miramistin-i-hlorgeksidin-mozhno-ispolzovat-vmesto-sanitajze...,4005.0,2020-09-05,longread,"Пандемия,Профилактика,Патогены,Вирусы",Проверка слуха
2,3745310,CLICKED,5fdba88d08d6334b9536bd71,491804,2023-07-01,491804,82267,6e97b34a-3a29-48e7-86b1-d63557cf298a,1,47,1978-01-07,Открываем окна: кому и как нужно проветривать квартиру,https://cuprum.media/science-answers/how-to-ventilate-the-apartment?from=avaapp,554.0,2020-12-17,longread,0,Ответили по науке
3,3745453,CLICKED,63ea42aaa6594aa87e07c4f2,65721,2023-07-01,65721,33336,bebd6f34-46ef-4f1c-8dbd-628bbfdae108,1,53,1972-02-03,"Правда ли, что лампы для сушки ногтей вызывают рак",https://cuprum.media/science-answers/nail-lamp-danger?from=avaapp,8032.0,2023-02-13,longread,"Вредное,Косметика,Булшит",Ответили по науке
4,3745512,CLICKED,608a88b3b587d22877730773,1281788,2023-07-01,1281788,336371,6cdfeb1f-8a4f-457c-a236-fc1d478ae716,2,74,1951-04-21,Можно ли забеременеть от двух мужчин одновременно,https://cuprum.media/stesnyayus-sprosit/superfecundation?from=avaapp,16787.0,2021-04-29,longread,"Репродукция,Булшит,Беременность",Стесняюсь спросить
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30835,9608721,CLICKED,63a9765bff95853c500cce62,97731,2024-07-30,97731,37822,09789d90-2f43-4c78-b29c-661ca30ca5c5,1,49,1976-05-09,Почему возникают ошибки в результатах анализов,https://cuprum.media/science-answers/lab-test-mistakes?from=avaapp,5907.0,2023-01-04,longread,"Анализы,Диагностика",Ответили по науке
30836,9608800,CLICKED,606dd15f70bc473f234d0092,1326732,2024-07-30,1326732,392685,2acb4b14-f65b-4170-9d24-d042a53280b2,2,21,2004-02-12,Полиамория и сологамия: какими бывают современные отношения,https://cuprum.media/lifestyle/romantic-relationship?from=avaapp,10308.0,2021-04-07,longread,Секс,Жизнь
30837,9609015,CLICKED,615c6f0cf2896444877b8b64,1430340,2024-07-30,1430340,517117,704e61a9-b817-4403-9ffd-0f5836755e8b,1,44,1981-01-29,Как бороться с пигментацией кожи,https://cuprum.media/lifestyle/skin-pigmentation?from=avaapp,4227.0,2021-10-05,longread,Кожа,Жизнь
30838,9609066,CLICKED,654612ff65e72be96c09461b,5716,2024-07-30,5716,23996,aae018e7-569b-4592-9696-09494d6c7970,1,46,1978-12-11,Как избавиться от второго подбородка,https://cuprum.media/science-answers/ubrat-vtoroy-podborodok?from=avaapp,741.0,2023-11-06,longread,"Красота,Образ жизни",Ответили по науке


In [4]:
data.drop(columns=["esb_ehr_id","patientnet_ehr_id"], inplace=True)
data.rename(columns={"пол":"gender", "возраст":"age"}, inplace=True)
data = data[data['tags'] != 'Секс']
data

,id,action_type,article_id,ehr_id,created_at,medialog_ehr_id,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title
0,3744933,CLICKED,61c03691a7568f6cda4a0c95,961420,2023-07-01,961420,2,35,1990-05-29,Чем опасна трипофобия,https://cuprum.media/science-answers/trypophobia?from=avaapp,795.0,2021-12-20,longread,0,Ответили по науке
1,3745298,CLICKED,5f22ce85e24e57105a57d2a5,100445,2023-07-01,100445,1,37,1987-09-26,Мирамистин и хлоргексидин можно использовать вместо санитайзера?,https://cuprum.media/proverka-sluha/miramistin-i-hlorgeksidin-mozhno-ispolzovat-vmesto-sanitajze...,4005.0,2020-09-05,longread,"Пандемия,Профилактика,Патогены,Вирусы",Проверка слуха
2,3745310,CLICKED,5fdba88d08d6334b9536bd71,491804,2023-07-01,491804,1,47,1978-01-07,Открываем окна: кому и как нужно проветривать квартиру,https://cuprum.media/science-answers/how-to-ventilate-the-apartment?from=avaapp,554.0,2020-12-17,longread,0,Ответили по науке
3,3745453,CLICKED,63ea42aaa6594aa87e07c4f2,65721,2023-07-01,65721,1,53,1972-02-03,"Правда ли, что лампы для сушки ногтей вызывают рак",https://cuprum.media/science-answers/nail-lamp-danger?from=avaapp,8032.0,2023-02-13,longread,"Вредное,Косметика,Булшит",Ответили по науке
4,3745512,CLICKED,608a88b3b587d22877730773,1281788,2023-07-01,1281788,2,74,1951-04-21,Можно ли забеременеть от двух мужчин одновременно,https://cuprum.media/stesnyayus-sprosit/superfecundation?from=avaapp,16787.0,2021-04-29,longread,"Репродукция,Булшит,Беременность",Стесняюсь спросить
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30833,9607899,CLICKED,60d599187d430d33ba5ab9c3,281078,2024-07-30,281078,1,43,1982-06-14,Существует ли синдром отмены сахара?,https://cuprum.media/science-answers/sugar-withdrawal-syndrome?from=avaapp,1347.0,2021-06-25,longread,0,Ответили по науке
30835,9608721,CLICKED,63a9765bff95853c500cce62,97731,2024-07-30,97731,1,49,1976-05-09,Почему возникают ошибки в результатах анализов,https://cuprum.media/science-answers/lab-test-mistakes?from=avaapp,5907.0,2023-01-04,longread,"Анализы,Диагностика",Ответили по науке
30837,9609015,CLICKED,615c6f0cf2896444877b8b64,1430340,2024-07-30,1430340,1,44,1981-01-29,Как бороться с пигментацией кожи,https://cuprum.media/lifestyle/skin-pigmentation?from=avaapp,4227.0,2021-10-05,longread,Кожа,Жизнь
30838,9609066,CLICKED,654612ff65e72be96c09461b,5716,2024-07-30,5716,1,46,1978-12-11,Как избавиться от второго подбородка,https://cuprum.media/science-answers/ubrat-vtoroy-podborodok?from=avaapp,741.0,2023-11-06,longread,"Красота,Образ жизни",Ответили по науке


In [5]:
def age_group(age):
    if age < 18:
        return 0 #'0-17'
    elif age < 30:
        return 1 #'18-29'
    elif age < 45:
        return 2 #'30-44'
    elif age < 60:
        return 3 #'45-59'
    else:
        return 4 #'60+'

data['age_group'] = data['age'].apply(age_group)
data

/tmp/ipykernel_3095/2221202540.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['age_group'] = data['age'].apply(age_group)


,id,action_type,article_id,ehr_id,created_at,medialog_ehr_id,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title,age_group
0,3744933,CLICKED,61c03691a7568f6cda4a0c95,961420,2023-07-01,961420,2,35,1990-05-29,Чем опасна трипофобия,https://cuprum.media/science-answers/trypophobia?from=avaapp,795.0,2021-12-20,longread,0,Ответили по науке,2
1,3745298,CLICKED,5f22ce85e24e57105a57d2a5,100445,2023-07-01,100445,1,37,1987-09-26,Мирамистин и хлоргексидин можно использовать вместо санитайзера?,https://cuprum.media/proverka-sluha/miramistin-i-hlorgeksidin-mozhno-ispolzovat-vmesto-sanitajze...,4005.0,2020-09-05,longread,"Пандемия,Профилактика,Патогены,Вирусы",Проверка слуха,2
2,3745310,CLICKED,5fdba88d08d6334b9536bd71,491804,2023-07-01,491804,1,47,1978-01-07,Открываем окна: кому и как нужно проветривать квартиру,https://cuprum.media/science-answers/how-to-ventilate-the-apartment?from=avaapp,554.0,2020-12-17,longread,0,Ответили по науке,3
3,3745453,CLICKED,63ea42aaa6594aa87e07c4f2,65721,2023-07-01,65721,1,53,1972-02-03,"Правда ли, что лампы для сушки ногтей вызывают рак",https://cuprum.media/science-answers/nail-lamp-danger?from=avaapp,8032.0,2023-02-13,longread,"Вредное,Косметика,Булшит",Ответили по науке,3
4,3745512,CLICKED,608a88b3b587d22877730773,1281788,2023-07-01,1281788,2,74,1951-04-21,Можно ли забеременеть от двух мужчин одновременно,https://cuprum.media/stesnyayus-sprosit/superfecundation?from=avaapp,16787.0,2021-04-29,longread,"Репродукция,Булшит,Беременность",Стесняюсь спросить,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30833,9607899,CLICKED,60d599187d430d33ba5ab9c3,281078,2024-07-30,281078,1,43,1982-06-14,Существует ли синдром отмены сахара?,https://cuprum.media/science-answers/sugar-withdrawal-syndrome?from=avaapp,1347.0,2021-06-25,longread,0,Ответили по науке,2
30835,9608721,CLICKED,63a9765bff95853c500cce62,97731,2024-07-30,97731,1,49,1976-05-09,Почему возникают ошибки в результатах анализов,https://cuprum.media/science-answers/lab-test-mistakes?from=avaapp,5907.0,2023-01-04,longread,"Анализы,Диагностика",Ответили по науке,3
30837,9609015,CLICKED,615c6f0cf2896444877b8b64,1430340,2024-07-30,1430340,1,44,1981-01-29,Как бороться с пигментацией кожи,https://cuprum.media/lifestyle/skin-pigmentation?from=avaapp,4227.0,2021-10-05,longread,Кожа,Жизнь,2
30838,9609066,CLICKED,654612ff65e72be96c09461b,5716,2024-07-30,5716,1,46,1978-12-11,Как избавиться от второго подбородка,https://cuprum.media/science-answers/ubrat-vtoroy-podborodok?from=avaapp,741.0,2023-11-06,longread,"Красота,Образ жизни",Ответили по науке,3


In [6]:
# Только положительный фидбек
clicked = data[data['action_type'] == 'CLICKED']

# Группируем
grouped = (
    clicked.groupby(['gender', 'age_group', 'article_id'])
    .size()
    .reset_index(name='clicks')
)

# Для удобства: получаем словарь топ-N статей на каждую группу
top_articles_by_segment = (
    grouped
    .sort_values(['gender', 'age_group', 'clicks'], ascending=[True, True, False])
    .groupby(['gender', 'age_group'])['article_id']
    .apply(lambda x: x.head(10).tolist())  # можно поменять число рекомендаций
    .to_dict()
)
top_articles_by_segment

{(1, 0): ['5fcf8c8860a3f827536dad61',
  '61a8e4c35a7b54125e39e744',
  '63763ca4bd190299f80b9e12',
  '63ab12c91c530bff010cf9a2',
  '63ca7bf0701b2742fd025762',
  '63caa8c5c67732cf86062b52',
  '63cab0cd085d9cdc94009d72',
  '65e70ca5a05adda29805ce7d',
  '5f2cf26bb45a9b7d7a63f0a4',
  '5f3a4783cc912654d436053d'],
 (1, 1): ['6001598cd96d8b7c8712ec51',
  '60ba3b22332a1c6822487722',
  '5f363ee1746d5c1b990fee6d',
  '5f4d0402091a3d52fc003014',
  '5fe1ee71ca1d6006e262be11',
  '61c5a180ea60a60eaf7d33ab',
  '6242edbbeb1c3c68d0152c02',
  '63a1c01ff8f8915c1b0510e2',
  '5fc779f97e9eef0cf9126411',
  '6128ea4059b68636c725cc86'],
 (1, 2): ['62bd9b574a4176582f017bb2',
  '5fe1ee71ca1d6006e262be11',
  '600992d6ad457f63cc193bb1',
  '63ad8d3d51bdac5ded02d4b2',
  '5fdfe5a410312331e524f0e1',
  '628cffd79a57867ed904f5a4',
  '60b74607911ec35036678cb3',
  '616ecd08d2bec01cda02fd72',
  '628b9967bbd64f605e639ae2',
  '5f3e713a11f6f83abf1ee6e1'],
 (1, 3): ['5f5b413b1b7818273828d682',
  '62a9f719ad21195610275454',
  '5f

In [7]:
# Шаг 1 — Оставляем только нужные столбцы, чтобы потом легко соединить
articles_info = clicked[['article_id', 'title', 'url']].drop_duplicates()

# Шаг 2 — Топ статей — словарь article_id + число кликов (как у тебя)
grouped = (
    clicked.groupby(['gender', 'age_group', 'article_id'])
    .size()
    .reset_index(name='clicks')
)

# Шаг 3 — Находим топ N article_id по группе
top_articles_by_segment = (
    grouped
    .sort_values(['gender', 'age_group', 'clicks'], ascending=[True, True, False])
    .groupby(['gender', 'age_group'])['article_id']
    .apply(lambda x: x.head(10).tolist())
    .to_dict()
)

# Шаг 4 — Функция рекомендаций — возвращает DataFrame с нужными полями
def recommend_by_demo(gender, age, N=10):
    group = age_group(age)
    ids = top_articles_by_segment.get((gender, group), [])[:N]
    # Соединяем с таблицей info, чтобы взять title и url
    recs = articles_info[articles_info['article_id'].isin(ids)]
    # Сохраняем тот же порядок, что и в топе
    recs['article_id'] = pd.Categorical(recs['article_id'], categories=ids, ordered=True)
    recs = recs.sort_values('article_id')
    return recs[['article_id', 'title', 'url']].reset_index(drop=True)

# Пример
recommendations = recommend_by_demo(gender=2, age=75)
recommendations


/tmp/ipykernel_3095/205444904.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recs['article_id'] = pd.Categorical(recs['article_id'], categories=ids, ordered=True)


,article_id,title,url
0,5fe1ee71ca1d6006e262be11,Как правильно вытирать попу,https://cuprum.media/stesnyayus-sprosit/toilet-paper?from=avaapp
1,60254b2820f8572dde564bf3,Мед полезнее сахара? Кому он может навредить?,https://cuprum.media/science-answers/honey-or-sugar?from=avaapp
2,6131f17257069656640271b3,Что за белые бугорки на лице и как от них избавиться?,https://cuprum.media/lifestyle/milium?from=avaapp
3,613b641ad0296d3df254ddf4,"Правда ли, что бегать по асфальту вредно",https://cuprum.media/proverka-sluha/running-on-asphalt?from=avaapp
4,628b9967bbd64f605e639ae2,Зачем отбеливать анус,https://cuprum.media/stesnyayus-sprosit/chocolate-eye?from=avaapp
5,5f257a920b4cc847170676da,Полип тела матки,https://cuprum.media/spravochnik/polip-tela-matki?from=avaapp
6,5f2cf26bb45a9b7d7a63f0a4,"Кажется, у меня проблемы с эрекцией. Это конец?",https://cuprum.media/ipohondrik/erectile-dysfunction?from=avaapp
7,5f31541aa6c981486a76a386,Сколько сексуальных партнеров за жизнь — это «нормально»?,https://cuprum.media/lifestyle/sex-eto-normalno?from=avaapp
8,5f363ee1746d5c1b990fee6d,Как часто нужно мыть голову?,https://cuprum.media/stesnyayus-sprosit/hair-washing?from=avaapp
9,5f3fe5035f43962ebf0e9565,Креатинин,https://cuprum.media/spravochnik/creatinine?from=avaapp


In [8]:
import pickle

# Сохраняем словарь
with open('top_articles_1.pkl', 'wb') as f:
    pickle.dump(top_articles_by_segment, f)

# Сохраняем таблицу
articles_info.to_csv('articles_info_1.csv', index=False)

In [9]:
# Проверка на всей истории
hits = 0
total_users = 0
N = 10

for _, row in clicked[['ehr_id', 'gender', 'age_group']].drop_duplicates().iterrows():
    seg_top = top_articles_by_segment.get((row.gender, row.age_group), [])[:N]
    user_clicks = clicked[clicked['ehr_id'] == row.ehr_id]['article_id'].unique()
    if any(a in seg_top for a in user_clicks):
        hits += 1
    total_users += 1

hit_rate = hits / total_users
print(f"HitRate@{N}: {hit_rate:.4f}")


HitRate@10: 0.0344


In [10]:
# Доля всех статей, попавших в топы
all_articles = set(clicked['article_id'])
top_articles = set(a for lst in top_articles_by_segment.values() for a in lst)
coverage = len(top_articles) / len(all_articles)
print(f"Coverage: {coverage:.4f}")

# Среднее количество кликов на статьи из топа
def avg_clicks_in_top(N=10):
    counts = []
    for _, row in clicked[['ehr_id', 'gender', 'age_group']].drop_duplicates().iterrows():
        seg_top = top_articles_by_segment.get((row.gender, row.age_group), [])[:N]
        user_clicks = clicked[clicked['ehr_id'] == row.ehr_id]['article_id']
        count_in_top = sum(a in seg_top for a in user_clicks)
        counts.append(count_in_top)
    return np.mean(counts)

print(f"Avg clicks in top-{N}: {avg_clicks_in_top():.2f}")


Coverage: 0.0232
Avg clicks in top-10: 0.05


# Итоговые метрики

- HitRate@10 = 3.44%
Это значит, что только примерно у каждого 30-го пользователя в его истории есть хотя бы одна статья из топ-10.
- Coverage = 2.32%
То есть в топах сегментов участвуют лишь ~2% всех статей. Это ожидаемо, потому что топ по кликам всегда концентрируется на очень небольшом количестве популярных материалов. Это ещё раз подтверждает, что метод работает как базовый fallback, а не как персонализатор.
- Avg clicks in top-10 = 0.05
В среднем пользователь кликнул только 0.05 статьи из своего сегментного топа за всё время (т.е. 1 клик на 20 пользователей). Это низко, но тоже ожидаемо для «холодного» и грубого сегментирования.

-------

Не супер, но для холодного старта пойдет.
Пересчет можно производить раз в 1-2 недели для пользователей на свежих данных и смотреть дальше

## Скрипт для записи топ категории в файл

In [ ]:
import pandas as pd
import pickle

# --- вспомогательная функция ---
def age_group(age: int) -> int:
    if age < 18:
        return 0  # '0-17'
    elif age < 30:
        return 1  # '18-29'
    elif age < 45:
        return 2  # '30-44'
    elif age < 60:
        return 3  # '45-59'
    else:
        return 4  # '60+'

# --- основной пайплайн ---
def build_and_save_top_articles(data: pd.DataFrame, top_n: int = 10,
                                dict_path: str = "top_articles.pkl",
                                csv_path: str = "top_articles_by_segment.csv"):
    """
    Строит топ статей по полу и возрастным группам, сохраняет в pkl и csv.
    """

    # Добавляем age_group
    data['age_group'] = data['age'].apply(age_group)

    # Только клики
    clicked = data[data['action_type'] == 'CLICKED']

    # Инфо о статьях
    articles_info = clicked[['article_id', 'title', 'url']].drop_duplicates()

    # Считаем клики по сегментам
    grouped = (
        clicked.groupby(['gender', 'age_group', 'article_id'])
        .size()
        .reset_index(name='clicks')
    )

    # Топ-N статей в каждом сегменте
    top_articles_by_segment = (
        grouped
        .sort_values(['gender', 'age_group', 'clicks'], ascending=[True, True, False])
        .groupby(['gender', 'age_group'])['article_id']
        .apply(lambda x: x.head(top_n).tolist())
        .to_dict()
    )

    # --- сохраняем словарь в pkl ---
    with open(dict_path, 'wb') as f:
        pickle.dump(top_articles_by_segment, f)

    # --- превращаем словарь в DataFrame для CSV ---
    rows = []
    for (gender, age_group), articles in top_articles_by_segment.items():
        for rank, article_id in enumerate(articles, start=1):
            rows.append({
                "gender": gender,
                "age_group": age_group,
                "rank": rank,
                "article_id": article_id
            })

    top_df = pd.DataFrame(rows)
    top_df = top_df.merge(articles_info, on="article_id", how="left")

    # --- сохраняем в CSV ---
    top_df.to_csv(csv_path, index=False)

    print(f"Топ сохранён: {csv_path}, словарь сохранён: {dict_path}")
    return top_df
